# Visualising rupture participation rates within fault systems.

We want to show how different parts of a fault system contribute to seismicity rate, with differnt levels of aggregation. This is known as Partipation rate (for sections/faults/etc)

We also want to do this for an entire aggregated set of compatible solutions OR for individual solutions.

### Definitions:
 - **participation rate** is the sum of rates for all ruptures involving a the given level of aggregation.
 - level of aggregation include **Named Fault**, **Parent Fault** (or *Fault*) or **Subsection** (or *Fault section*)
 - where a **Named Fault** has one or more **Parent Fault**s that each have at least two **Subsection**  

Steps:
 - select a suitable InversionSolution or Composite Solution as the data source

In [1]:
#import pathlib
import json
import nzshm_model as nm
import geopandas as gpd
import solvis

# from solvis.fault_system_solution_helper import FaultSystemSolutionHelper
from solvis_graphql_api.color_scale import ColourScaleNormaliseEnum, get_colour_values, get_colour_scale
from ipyleaflet import Map, GeoJSON, LegendControl, FullScreenControl, Popup, ScaleControl, WidgetControl
from ipywidgets import HTML

from solvis.filter import FilterRuptureIds, FilterSubsectionIds

warning openquake module dependency not available, maybe you want to install
                with nzshm-model[openquake]


Running without `toshi` options


In [2]:
# OPTIONS
SINGLE_SOLUTION = False
SECTION_RATE = False # otherwise Parent rates

if SINGLE_SOLUTION:
    solution = solvis.InversionSolution.from_archive("NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTEzMTM0.zip")
else:
    current_model = nm.get_model_version("NSHM_v1.0.4")
    slt = current_model.source_logic_tree
    csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
    solution = csol._solutions['CRU']

## Choose some faults and get their unique rupture IDs

In [3]:
TARGET_FAULTS = ['Opotiki 3', 'Calypso 2','Wellington Hutt Valley: 1', 'Wellington Hutt Valley: 2', 'Wairarapa: 1'] #, 'Awatere: Southwest', 'Wairarapa 1', 'Awatere: Southwest', 'Pokeno', 'BooBoo', "Masterton
ruptures = FilterRuptureIds(solution)\
    .for_parent_fault_names(TARGET_FAULTS)\
    .for_magnitude(5, 8.2)

# get rupture fault sections (rs) with rates for those ruptures
df0 = solution.rs_with_rupture_rates
rupture_sections_df = df0[df0["Rupture Index"].isin(ruptures)]
print(f' {len(ruptures)} unique ruptures...')

 337 unique ruptures...


In [4]:
section_count = len(rupture_sections_df['section'].unique())
# print(f'the faults in {TARGET_FAULTS} have:')
print(f' {section_count} unique fault_sections...')
print()
rupture_ids = list(rupture_sections_df["Rupture Index"].unique())

 492 unique fault_sections...



### calculate participation rate series

In [5]:
# get the participation rate for each subsection in the selected ruptures
subsections = FilterSubsectionIds(solution).for_rupture_ids(rupture_ids)
rate_column = "Annual Rate" if isinstance(solution, solvis.InversionSolution) else "rate_weighted_mean"

#parent fault participation
rs_df = solution.rs_with_rupture_rates [solution.rs_with_rupture_rates.section.isin(subsections)]
pf_df = rs_df.join(solution.fault_sections[['ParentID']], on='section')
fault_rates = pf_df[["Rupture Index", "ParentID", rate_column]]
parent_rates = fault_rates.groupby("ParentID").agg('sum')[rate_column]
parent_rates.head()

ParentID
2     0.000229
6     0.000426
7     0.000743
27    0.000096
29    0.000189
Name: rate_weighted_mean, dtype: float64

In [7]:
# let's get the participation rate for each subsection in the selected ruptures
section_rates = rs_df[["Rupture Index", "section", rate_column]].groupby("section").agg('sum')[rate_column]

In [ ]:
# check sum of rates matches
if not parent_rates.sum() == section_rates.sum():
    print(parent_rates.sum())
    print(section_rates.sum())

### Style the geojson using a color scale

In [6]:
surfaces = solution.fault_surfaces()
surfaces = surfaces[surfaces["FaultID"].isin(subsections)]

#join the rate series from earlier
if SECTION_RATE:
    rate_series = section_rates
    surfaces = surfaces.join(rate_series, on='FaultID', how='outer')
else:
    rate_series = parent_rates 
    surfaces = surfaces.join(rate_series, on='ParentID', how='outer')
surfaces.rename(columns={rate_series.name: "annual_rate"}, inplace=True)

fault_sections_gdf = gpd.GeoDataFrame(surfaces)

# create the colour scale from our rate series
color_values = get_colour_values(
                color_scale="inferno",
                color_scale_vmax=rate_series.max(),
                color_scale_vmin=rate_series.min(),
                color_scale_normalise=ColourScaleNormaliseEnum.LOG.value,
                values=tuple(rate_series.tolist()),
            )

rate_ids = rate_series.index.tolist()
data = json.loads(fault_sections_gdf.to_json())

# merge the styling with the geojson

for feature in data["features"]:
    try:
        if SECTION_RATE:
            color = color_values[rate_ids.index(feature['properties']['FaultID'])]
        else:
            color = color_values[rate_ids.index(feature['properties']['ParentID'])]
    except (ValueError):
        print(f"warning no rate found for index: {feature['properties']['FaultID']} faultname: `{feature['properties']['FaultName']}`")
        color = 'cyan' 
        
    if feature['properties']['DipDeg'] == 90: 
        # these vertical sections are much harder to see, so color the line and make it heavier 
        line_color = color
        weight = 3
    else:
        line_color = "black"
        weight = 1
        
    feature["properties"]["style"] = {
        "color": line_color,
        "weight": weight,
        "fillColor": color,
        "fillOpacity": 1,
    }

### build a legend

In [7]:
# from SGI
cs = get_colour_scale(
        color_scale="inferno",
        color_scale_normalise=ColourScaleNormaliseEnum.LOG.value, 
        vmax=rate_series.max(),
        vmin= rate_series.min())

color_map = dict(zip(reversed(cs.color_map.levels), reversed(cs.color_map.hexrgbs)))


### Display with ipyleaflet

In [8]:
center = [-41.5, 175]
zoom = 7
map = Map(center=center, zoom=zoom)

section_info = HTML()
section_info.value = "<b>Section Detail</b><br/><p>hover over fault sections for more details.</p>"
widget_control = WidgetControl(widget=section_info, position='topright')
legend = LegendControl(color_map, title="Rupture Rate/yr", position="topright")

def on_hover_callback(event, feature, properties, id):
    section_info.value = f"<b>{properties['FaultName']}</b>"
    section_info.value += "<br />"
    section_info.value += f"Dip: {properties['DipDeg']}</br>"
    section_info.value += f"Rake: {properties['Rake']}</br>"
    section_info.value += f"Lower depth: {round(properties['LowDepth'],3)}</br>"    
    section_info.value += f"Participation rate: {properties['annual_rate']:.2E}</br>" 
    
g = GeoJSON(data=data, 
            hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.1})
g.on_hover(on_hover_callback)

map.add(g)
map.add(FullScreenControl())
map.add(widget_control)
map.add(legend)
map.add(ScaleControl(position='bottomleft', max_width=250))

#render the map ...
map

Map(center=[-41.5, 175], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…